In [8]:
#!pip install pandas

## Importación de Librerias

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Carga de Datos
Se trabajará con 2 datasets, uno con datos referidos a las viviendas y otro referido a las personas habitantes de la vivienda.

In [10]:
# Carga de Datos de vivienda.
df_hogares = pd.read_csv('../data/raw/EPH_usu_hogar_tic_t418.txt', delimiter=';')

# Carga de Datos de personas.
df_personas = pd.read_csv('../data/raw/EPH_usu_Indiv_tic_t418.txt', delimiter=';')

## Impresión de Datos de viviendas

In [11]:

print(df_hogares.head())

                          CODUSU  NRO_HOGAR  REALIZADA  ANO4  TRIMESTRE  \
0  TQRMNOPPQHJMKMCDEFIAH00622969          1          1  2018          4   
1  TQRMNOPPQHJMLUCDEFIAH00622993          1          1  2018          4   
2  TQRMNOPPQHMMLUCDEFIAH00594749          1          1  2018          4   
3  TQRMNOPPRHJMLUCDEFIAH00623008          1          1  2018          4   
4  TQRMNOPPSHJMKMCDEFIAH00623022          1          1  2018          4   

   AGLOMERADO  POND_TIC  IH_II_01  IH_II_02  
0           2       539         1         1  
1           2       545         1         1  
2           2       612         1         1  
3           2       545         1         1  
4           2       539         1         1  


## Impresión de Datos de personas

In [12]:
print(df_personas.head())

                          CODUSU  NRO_HOGAR  COMPONENTE  ANO4  TRIMESTRE  \
0  TQRMNOPPQHJMKMCDEFIAH00622969          1           1  2018          4   
1  TQRMNOPPQHJMKMCDEFIAH00622969          1           2  2018          4   
2  TQRMNOPPQHJMKMCDEFIAH00622969          1           3  2018          4   
3  TQRMNOPPQHJMLUCDEFIAH00622993          1           1  2018          4   
4  TQRMNOPPQHJMLUCDEFIAH00622993          1           2  2018          4   

   AGLOMERADO  POND_TIC  RESPONDENTE  IP_III_04  IP_III_05  IP_III_06  
0           2       539          1.0          1          1          1  
1           2       539          1.0          1          1          1  
2           2       539          1.0          1          1          1  
3           2       545          2.0          1          1          1  
4           2       545          2.0          1          1          1  


## Limpieza y preparación de Datos

In [13]:
# Selección de filas relevantes para el análisis de hogares (solo trabajaremos con datos de Rio Grande y Ushuaia)
df_hogartdf = df_hogares[df_hogares['AGLOMERADO'] == 31]

# Selección de columnas relevantes para el análisis de hogares
hogar_cols = ['CODUSU', 'NRO_HOGAR', 'IH_II_01', 'IH_II_02']
df_hogartdf = df_hogartdf[hogar_cols]

# Renombrando columnas para mayor claridad
df_hogartdf.columns = ['codigo_vivienda', 'nro_hogar', 'tiene_computadora', 'acceso_internet']

# Eliminación de filas con valores faltantes
df_hogartdf = df_hogartdf.dropna()

# Vista previa del dataset de viviendas
df_hogartdf.head()

,codigo_vivienda,nro_hogar,tiene_computadora,acceso_internet
12285,TQRMNOPPQHJMKPCDEIHJF00626473,1,1,1
12286,TQRMNOPPQHKMKNCDEIHJF00620952,1,1,1
12287,TQRMNOPPRHJMKPCDEIHJF00626438,1,1,1
12288,TQRMNOPPRHKNKQCDEIHJF00620859,1,1,1
12289,TQRMNOPPSHJMKMCDEIHJF00626453,1,1,1


In [14]:
# Selección de columnas relevantes para el análisis de personas (solo trabajaremos con datos de Rio Grande y Ushuaia)
df_personatdf = df_personas[df_personas['AGLOMERADO'] == 31]

# Selección de columnas relevantes para el análisis de personas
persona_cols = ['CODUSU', 'NRO_HOGAR', 'IP_III_04', 'IP_III_05', 'IP_III_06']
df_personatdf = df_personatdf[persona_cols]

# Renombrando columnas para mayor claridad
df_personatdf.columns = ['codigo_vivienda', 'nro_hogar', 'uso_internet', 'uso_computadora', 'uso_movil']

# Eliminación de filas con valores faltantes
df_personatdf = df_personatdf.dropna()

# Vista previa del dataset de personas
df_personatdf.head()

,codigo_vivienda,nro_hogar,uso_internet,uso_computadora,uso_movil
36805,TQRMNOPPQHJMKPCDEIHJF00626473,1,1,1,1
36806,TQRMNOPPQHJMKPCDEIHJF00626473,1,1,1,1
36807,TQRMNOPPQHKMKNCDEIHJF00620952,1,1,1,1
36808,TQRMNOPPQHKMKNCDEIHJF00620952,1,1,1,1
36809,TQRMNOPPQHKMKNCDEIHJF00620952,1,1,1,2


## Análisis Descriptivo de los Datos

In [15]:
# Proporción de hogares con acceso a internet
print("\nProporción de hogares con acceso a internet: (1 SI tiene acceso, 2 NO tiene acceso)")
print(df_hogartdf['acceso_internet'].value_counts(normalize=True))

# Proporción de uso de internet entre personas
print("\nProporción de personas que usan internet: (1 SI usa internet, 2 NO usa internet)")
print(df_personatdf['uso_internet'].value_counts(normalize=True))


Proporción de hogares con acceso a internet: (1 SI tiene acceso, 2 NO tiene acceso)
acceso_internet
1    0.894472
2    0.105528
Name: proportion, dtype: float64

Proporción de personas que usan internet: (1 SI usa internet, 2 NO usa internet)
uso_internet
1    0.913462
2    0.086538
Name: proportion, dtype: float64


## Construcción del Modelo de Regresión Logística

In [16]:
# Unir datos de hogares y personas
df_hogares_personas = pd.merge(df_hogartdf, df_personatdf, on=['codigo_vivienda', 'nro_hogar'])

# Variables independientes y dependientes
X = df_hogares_personas[['tiene_computadora', 'uso_computadora', 'uso_movil']]
y = df_hogares_personas['acceso_internet']

# Convertir variables categóricas a numéricas si es necesario
X = pd.get_dummies(X, drop_first=True)

## Resguardo de BBDD procesada

In [17]:
# Se guarga una copia del dataset de hogares y personas ya procesada en la carpeta processed.
df_hogares_personas.to_csv("../data/processed/EPH_usu_hogar_Indiv_tic_t418_V2.txt")

## División de Datos en Conjuntos de Entrenamiento y Prueba

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Entrenamiento del Modelo

In [19]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

## Evaluación del Modelo

In [20]:
y_pred = model.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9418604651162791

Confusion Matrix:
[[305  14]
 [  6  19]]

Classification Report:
              precision    recall  f1-score   support

           1       0.98      0.96      0.97       319
           2       0.58      0.76      0.66        25

    accuracy                           0.94       344
   macro avg       0.78      0.86      0.81       344
weighted avg       0.95      0.94      0.95       344

